In [34]:
import pandas as pd
import numpy as np

xRaw = pd.read_csv("cse575_proj_data/x_raw.csv", sep=',',header=None)
yRaw = pd.read_csv("cse575_proj_data/y_raw.csv", sep=',',header=None)

#print(xRaw.head().values)
#print(yRaw.head().values)

In [9]:
xnplist = [np.array(xi) for xi in xRaw.values]
ylist = [y[0] for y in yRaw.values]

X = np.stack(xnplist)
y = np.array(ylist)

print(X.shape)
print(y.shape)

(5043, 55)
(5043,)


In [39]:
import pandas as pd

RAW = pd.read_csv("output.csv")

In [43]:
ylist = RAW['koi_disposition'].to_list()
y = np.array(ylist)

Xdf = RAW.drop(columns=['koi_disposition'])
xnplist = [np.array(xi) for xi in Xdf.values]
X = np.stack(xnplist)

print(X.shape)
print(y.shape)

(5043, 25)
(5043,)


In [48]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn import svm

clf1 = LogisticRegression(max_iter = 1000,random_state=1)
clf2 = RandomForestClassifier(n_estimators=50, random_state=1)
clf3 = GaussianNB()
clf4 = svm.SVC(probability = True)

eclf1 = VotingClassifier(estimators=[('lr', clf1), ('rf', clf2), ('gnb', clf3), ('svm', clf4)], voting='soft')

In [45]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

eclf1 = eclf1.fit(X_train, y_train)

In [51]:
from sklearn import metrics
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, random_state=0, shuffle = True)

avgs = []
macrof1s = []
microf1s = []
weightedf1s = []
tps = []
fps = []
fns = []
tns = []


for train_index, test_index in kf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    eclf1 = eclf1.fit(X_train, y_train)
    
    y_pred = eclf1.predict(X_test)
    avgs.append(metrics.accuracy_score(y_test, y_pred))
    macrof1s.append(f1_score(y_test, y_pred, average='macro'))
    microf1s.append(f1_score(y_test, y_pred, average='micro'))
    weightedf1s.append(f1_score(y_test, y_pred, average='weighted'))
    tps.append(confusion_matrix(y_test, y_pred)[0][0])
    fps.append(confusion_matrix(y_test, y_pred)[0][1])
    fns.append(confusion_matrix(y_test, y_pred)[1][0])
    tns.append(confusion_matrix(y_test, y_pred)[1][1])
    
    

print(sum(avgs)/len(avgs))
print(sum(macrof1s)/len(avgs))
print(sum(microf1s)/len(avgs))
print(sum(weightedf1s)/len(avgs))
print(sum(tps)/len(avgs))
print(sum(fps)/len(avgs))
print(sum(fns)/len(avgs))
print(sum(tns)/len(avgs))

0.9436830430883949
0.9425204870013207
0.9436830430883949
0.943869803704277
547.6
40.0
16.8
404.2


In [47]:
from sklearn import metrics

#evaluate model
y_pred = eclf1.predict(X_test)


print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

print(f1_score(y_test, y_pred, average='macro'))
print(f1_score(y_test, y_pred, average='micro'))
print(f1_score(y_test, y_pred, average='weighted'))

Accuracy: 0.9438202247191011
[[821  56]
 [ 29 607]]
              precision    recall  f1-score   support

         0.0       0.97      0.94      0.95       877
         1.0       0.92      0.95      0.93       636

    accuracy                           0.94      1513
   macro avg       0.94      0.95      0.94      1513
weighted avg       0.94      0.94      0.94      1513

0.9426733762062751
0.9438202247191011
0.9439649205595044
